In [ ]:
import numpy as np
import os
from scipy.signal import find_peaks, butter, filtfilt
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

def preprocess_ppg(ppg_data, distance=116, prominence=0.5):
    """
    PPG 데이터 전처리 및 피크 감지
    - 필터링 후 find_peaks 적용
    - prominence 값을 조정하여 확실한 피크만 감지
    """
    def butter_lowpass_filter(data, cutoff=5, fs=100, order=3):
        nyquist = 0.5 * fs
        normal_cutoff = cutoff / nyquist
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        return filtfilt(b, a, data)

    filtered_ppg = butter_lowpass_filter(ppg_data)
    peaks, _ = find_peaks(filtered_ppg, distance=distance, prominence=prominence)
    
    return peaks, filtered_ppg

def stretch_signal(segment, target_length=2000):
    """
    주어진 PPG 신호 세그먼트를 2000 샘플로 늘리는 함수
    - Cubic Interpolation(3차 스플라인 보간) 사용
    """
    original_length = len(segment)

    # 기존 데이터 포인트 인덱스를 [0, 1] 범위로 정규화
    x_old = np.linspace(0, 1, original_length)
    x_new = np.linspace(0, 1, target_length)
    
    # 3차 스플라인 보간 적용하여 신호 확장
    interpolator = interp1d(x_old, segment, kind='cubic')  
    stretched_signal = interpolator(x_new)
    
    return stretched_signal

def segment_ppg_resampling(ppg_data, peaks, target_peak_count=8, target_length=2000, save_dir=None):
    """
    PPG 데이터를 n개 피크 단위로 분할한 후 2000 샘플로 리샘플링(Stretching)
    - 부족한 부분을 단순 패딩 대신 리샘플링하여 데이터 왜곡 방지
    """
    segments = []

    # 저장 폴더 생성 (필요 시)
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)

    for i in range(0, len(peaks) - target_peak_count, target_peak_count):  
        start_idx = peaks[i]
        end_idx = peaks[i + target_peak_count - 1]  # n번째 피크 위치
        segment = ppg_data[start_idx:end_idx]

        # 리샘플링(Stretching) 적용
        segment = stretch_signal(segment, target_length=target_length)

        # 저장
        segments.append(segment)

        if save_dir:
            file_name = os.path.join(save_dir, f"segment_{i//target_peak_count}.npy")
            np.save(file_name, segment)

    return np.array(segments)

# PPG 데이터 로드
ppg_data = np.load(r"C:\Users\USER\Desktop\PPG\PPG_graph\npy\normal_10.npy")  

# 피크 찾기 (필터링 추가 후 개선된 피크 감지 적용)
peaks, filtered_ppg = preprocess_ppg(ppg_data)

# 개선된 방법으로 데이터 분할 및 저장 (n개 피크 단위 & 리샘플링 적용)
save_dir = r"C:\Users\USER\Desktop\PPG\PPG_graph\cutByPeak_8\n10"
segments = segment_ppg_resampling(ppg_data, peaks, target_peak_count=8, save_dir=save_dir)

# 결과 출력
print(f"✅ 총 {len(segments)}개의 PPG 세그먼트가 {save_dir}에 저장되었습니다.")
